<a href="https://colab.research.google.com/github/Sanjeevp-07/SolarXchange_ML/blob/main/SolarXchange_fec_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopy scikit-learn pandas


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
buyers = pd.read_csv('/content/drive/MyDrive/SolarXchange/greater_noida_buyers.csv')

sellers = pd.read_csv('/content/drive/MyDrive/SolarXchange/greater_noida_sellers.csv')

In [ ]:
buyers.head()

,buyer_id,latitude,longitude,energy_needed_kwh
0,B001,28.473312,77.488659,3.47
1,B002,28.478417,77.516777,12.59
2,B003,28.481301,77.507323,10.53
3,B004,28.470388,77.496515,3.98
4,B005,28.476641,77.519512,13.48


In [ ]:
sellers.head()

,seller_id,latitude,longitude,energy_available_kwh,price_per_kwh
0,S001,28.479367,77.489846,18.16,5.38
1,S002,28.473017,77.499794,16.11,4.27
2,S003,28.480877,77.500573,15.46,5.42
3,S004,28.489630,77.495977,15.54,5.31
4,S005,28.472058,77.502387,10.39,3.89


In [ ]:
import pandas as pd
from geopy.distance import geodesic

buyers_df = pd.read_csv('/content/drive/MyDrive/SolarXchange/greater_noida_buyers.csv')

sellers_df = pd.read_csv('/content/drive/MyDrive/SolarXchange/greater_noida_sellers.csv')

def recommend_seller(buyer, sellers_df):
    recommendations = []

    for _, seller in sellers_df.iterrows():

        dist = geodesic((buyer['latitude'], buyer['longitude']), (seller['latitude'], seller['longitude'])).meters

        # Energy Match Score
        energy_score = min(buyer['energy_needed_kwh'], seller['energy_available_kwh']) / buyer['energy_needed_kwh']

        max_price = sellers_df['price_per_kwh'].max()
        price_score = 1 - (seller['price_per_kwh'] / max_price)

        total_score = (0.4 * energy_score) + (0.3 * price_score) + (0.3 * (1 / (1 + dist / 100)))

        recommendations.append({
            'seller_id': seller['seller_id'],
            'distance_m': round(dist, 2),
            'price_per_kwh': seller['price_per_kwh'],
            'energy_match_score': round(energy_score, 2),
            'final_score': round(total_score, 4)
        })

    sorted_recommendations = sorted(recommendations, key=lambda x: x['final_score'], reverse=True)
    return pd.DataFrame(sorted_recommendations)

best_sellers = recommend_seller(buyers_df.iloc[0], sellers_df)

best_sellers.head()

,seller_id,distance_m,price_per_kwh,energy_match_score,final_score
0,S043,72.25,4.28,1.0,0.6399
1,S011,544.42,3.64,1.0,0.5473
2,S059,592.04,3.68,1.0,0.5419
3,S061,684.60,3.62,1.0,0.5401
4,S071,818.51,3.67,1.0,0.5317
